<a href="https://colab.research.google.com/github/yusam/mead2/blob/main/autos_de_infracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carregando bibliotecas

In [251]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob

### Montando drive do Google Drive

In [252]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Carregando dataset


In [253]:
infracao_aux_2018 = []
infracao_aux_2019 = []
infracao_aux_2020 = []


#le todos os arquivos de 2020
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2020/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2020.append(infracao_aux)

    
   
#le todos os arquivos de 2019
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2019/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    infracao_aux.columns=["tipo_infracao",
                          "descricao",
                          "tipo_infrator",
                          "tipo_veiculo",
                          "data_cometimento",
                          "hora_cometimento",
                          "local_rodovia",
                          "local_km",
                          "local_referencia",
                          "local_complemento",
                          "latitude",
                          "longitude",
                          "gravidade"]
    infracao_aux_2019.append(infracao_aux)


#le todos os arquivos de 2018
for doc in glob.glob("/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/2018/*"):
    infracao_aux = pd.read_csv(doc,sep=";",encoding="latin-1",low_memory=False)
    if infracao_aux.shape[1] == 6:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",]
    else:
        infracao_aux.columns=["tipo_infracao",
                              "descricao",
                              "tipo_infrator",
                              "tipo_veiculo",
                              "data_cometimento",
                              "hora_cometimento",
                              "local_rodovia",
                              "local_km",
                              "local_referencia",
                              "local_complemento",
                              "latitude",
                              "longitude",
                              "gravidade"]
    infracao_aux_2018.append(infracao_aux)    

In [254]:
## juntando dataframes de anos difrentes
infracao = pd.concat(infracao_aux_2020,ignore_index = True)
infracao = pd.concat(infracao_aux_2019,ignore_index = True)
infracao = pd.concat(infracao_aux_2018,ignore_index = True)

infracao.head().T


,0,1,2,3,4
tipo_infracao,7455-0,7455-0,6769-0,7285-0,7579-0
descricao,Transitar em velocidade superior à máxima per...,Transitar em velocidade superior à máxima per...,Conduzir veíc c/ defeito no sist de iluminaçã...,Em movimento deixar de manter a placa traseir...,Recusar-se a ser submetido a teste exame clín...
tipo_infrator,Condutor,Condutor,Proprietário,Condutor,Condutor
tipo_veiculo,AUTOMOVEL,AUTOMOVEL,Automóvel,Automóvel,Automóvel
data_cometimento,01/07/2018,01/07/2018,01/07/2018,01/07/2018,01/07/2018
hora_cometimento,00:01,00:02,00:02,00:02,00:03
local_rodovia,NaN,NaN,NaN,NaN,NaN
local_km,NaN,NaN,NaN,NaN,NaN
local_referencia,NaN,NaN,NaN,NaN,NaN
local_complemento,NaN,NaN,NaN,NaN,NaN


In [255]:
## verificando tipo de variaveis
infracao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395758 entries, 0 to 395757
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   tipo_infracao      395758 non-null  object
 1   descricao          395758 non-null  object
 2   tipo_infrator      395758 non-null  object
 3   tipo_veiculo       393141 non-null  object
 4   data_cometimento   395758 non-null  object
 5   hora_cometimento   395758 non-null  object
 6   local_rodovia      294256 non-null  object
 7   local_km           119404 non-null  object
 8   local_referencia   124419 non-null  object
 9   local_complemento  69511 non-null   object
 10  latitude           120109 non-null  object
 11  longitude          120109 non-null  object
 12  gravidade          294256 non-null  object
dtypes: object(13)
memory usage: 39.3+ MB


In [256]:
## verificando qtd de linhas e colunas
infracao.shape

(395758, 13)

### Limpeza dos dados

#### Tratando valores duplicados

In [257]:
## verificando valores duplicados
infracao.duplicated().value_counts()

False    371468
True      24290
dtype: int64

In [258]:
## dropando valores duplicados
infracao.drop_duplicates(inplace=True)
infracao.shape

(371468, 13)

#### Normalizando dados

In [259]:
## padronizando tipo infracao
infracao.loc[:,"tipo_infracao"] = infracao.loc[:,"tipo_infracao"].str.strip()
infracao.tipo_infracao.value_counts()

7455-0    179252
5681-0     30438
7463-0     21311
6769-0     19510
7633-2     11471
           ...  
6556-2         1
6009-2         1
6017-5         1
6459-2         1
6718-0         1
Name: tipo_infracao, Length: 235, dtype: int64

In [260]:
## padronizando tipo infrator
infracao.loc[:,"tipo_infrator"] = (infracao.loc[:,"tipo_infrator"]
                                   .str.strip()
                                   .str.replace("Condutor","CONDUTOR")
                                   .str.replace("(Proprietário|ProprietÃ¡rio)","PROPRIETARIO")
                                   .str.replace("(Pessoa Jurídica|Pessoa JurÃ\xaddica)","PESSOA_JURIDICA")
                                   .str.replace("(Pessoa Física|Pessoa FÃ\xadsica)","PESSOA_FISICA")
                                   .str.replace("Servidor Público","SERVIDOR_PUBLICO")
                                   )
infracao.tipo_infrator.value_counts()

CONDUTOR            341541
PROPRIETARIO         29906
SERVIDOR_PUBLICO        11
PESSOA_JURIDICA          5
PESSOA_FISICA            5
Name: tipo_infrator, dtype: int64

In [261]:
## padronizando gravidade da infracao
infracao.loc[:,"gravidade"] = (infracao.loc[:,"gravidade"]
                               .str.strip()
                               .str.replace("(MÃ©dia)","MEDIA")
                               .str.replace("GravÃ\xadssima","GRAVISSIMA")
                               .str.replace("Leve","LEVE")
                               .str.replace("Grave","GRAVE")
                               )
infracao.gravidade.value_counts()

MEDIA         169829
GRAVE          44438
GRAVISSIMA     36334
LEVE           31616
Name: gravidade, dtype: int64

In [262]:
## padronizando nome tipo de veiculo
infracao.loc[:,"tipo_veiculo"] = (infracao.loc[:,"tipo_veiculo"]
                                  .str.strip()
                                  .str.replace("(autom|Autom|AUTOM|0    AUTOM)....*","AUTOMOVEL")
                                  #.str.replace("0*","AUTOMOVEL")
                                  .str.replace("(MOTOCI|Motoci).*","MOTOCICLETA")
                                  .str.replace("(Camio|CAMIO).*","CAMIONETA")
                                  .str.replace("(Caminho|CAMINHO).*","CAMINHONETE")
                                  .str.replace("(CAMINH|Caminh)[^o|^O].[^\s]*|C[\.]","CAMINHAO")
                                  .str.replace("(\sTRATOR|\sTrator)","_TRATOR")
                                  .str.replace("(UTILI|Utili).*","UTILITARIO")
                                  .str.replace("(MOTON|Moton).*","MOTONETA")
                                  .str.replace("(.NIBU|.nibu|Ã\x94).*","ONIBUS")
                                  .str.replace("(MICR|Micr|Ã\x94).*","MICROONIBUS")
                                  .str.replace("(TRIC|Tric).*","TRICICLO")
                                  .str.replace("[TRIC|Tric](iclom|ICLOM).*","CICLOMOTOR")
                                  .str.replace("(Rebo|REBO).*","REBOQUE")
                                  .str.replace("(Semi|SEMI|S\.)(-|)","SEMI")
                                  .str.replace("(MOTOR CASA|MOTOR-CASA)","TRAILER")
                                  .str.replace("Chassi Plataforma","CHASSI_PLATAFORMA")
                                  .str.replace("Charrete","CHARRETE")
                                  .str.replace("Bicicleta","BICICLETA")
                                  .str.replace("Bonde","BONDE")
                                  .str.replace("Tr Rodas","TRANSP_RODAS")
                                  .str.replace("Tr Misto","TRANSP_MISTO")
                                  .str.replace("Tr Esteiras","TRANSP_ESTEIRAS")
                                  .replace("-",np.nan)
                                  .replace("INEXISTENTE",np.nan)
                                  )
infracao.tipo_veiculo.value_counts()

AUTOMOVEL            256721
CAMINHONETE           30571
CAMIONETA             22666
MOTOCICLETA           21771
UTILITARIO            12954
CAMINHAO               8342
CAMINHAO_TRATOR        4661
MICROONIBUS            2283
null                   2093
SEMIREBOQUE            1999
ONIBUS                 1744
MOTONETA               1464
REBOQUE                 437
CICLOMOTOR               33
TRICICLO                 20
TRAILER                  10
CHASSI_PLATAFORMA         4
TRANSP_RODAS              2
BONDE                     2
BICICLETA                 1
CHARRETE                  1
TRANSP_ESTEIRAS           1
TRANSP_MISTO              1
Name: tipo_veiculo, dtype: int64

#### Tratando valores nulos

In [263]:
## verificando a quantidade de nulos
infracao.isna().sum().sort_values()

tipo_infracao             0
descricao                 0
tipo_infrator             0
data_cometimento          0
hora_cometimento          0
tipo_veiculo           3687
local_rodovia         89251
gravidade             89251
local_referencia     252878
latitude             257196
longitude            257196
local_km             257528
local_complemento    304851
dtype: int64

In [264]:
## dropando colunas nulas não utilizadas
infracao = infracao.drop(columns=['longitude','latitude','local_complemento','local_referencia','local_km','local_rodovia'])
infracao.isnull().sum().sort_values()

tipo_infracao           0
descricao               0
tipo_infrator           0
data_cometimento        0
hora_cometimento        0
tipo_veiculo         3687
gravidade           89251
dtype: int64

In [265]:
## tratando valores nulos
## utilizando a moda para os tipos de veiculos 
tipo_veiculo_moda = infracao['tipo_veiculo'].mode().to_string()
print(type(tipo_veiculo_moda))
infracao['tipo_veiculo'].fillna(value = tipo_veiculo_moda, inplace=True)
infracao.isnull().sum().sort_values()

<class 'str'>


tipo_infracao           0
descricao               0
tipo_infrator           0
tipo_veiculo            0
data_cometimento        0
hora_cometimento        0
gravidade           89251
dtype: int64

### Gerando arquivo tratado

In [266]:
## exportando resultado
infracao.to_csv('/content/drive/MyDrive/Colab Notebooks/autos_de_infracao/infracoes_2018_2020_tratado.csv', sep=';',encoding="latin-1")